In [1]:
#comment this if you are not using AIT proxy...
import os
os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [2]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import torch.nn.functional as F
import random
import os
import numpy as np
import utils
# Set the random seed for reproducible experiments
random.seed(230)
torch.manual_seed(230)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Pruning

Reference : https://pytorch.org/tutorials/intermediate/pruning_tutorial.html#iterative-pruning

In [4]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

## 1.Load Dataset

In [6]:
from datasets import load_dataset

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
task_name = "sst2"
datasets = load_dataset("glue",task_name)
datasets

Found cached dataset glue (/home/todsavadt/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 1185.50it/s]


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

## 2. Preprocessing

In [9]:
student = "distilroberta-base"
teacher = "textattack/roberta-base-SST-2"

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(student)

In [11]:
# Labels
if task_name is not None:
    is_regression = task_name == "stsb"
    if not is_regression:
        label_list = datasets["train"].features["label"].names
        num_labels = len(label_list)
    else:
        num_labels = 1
else:
    # Trying to have good defaults here, don't hesitate to tweak to your needs.
    is_regression = datasets["train"].features["label"].dtype in ["float32", "float64"]
    if is_regression:
        num_labels = 1
    else:
        # A useful fast method:
        # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.unique
        label_list = datasets["train"].unique("label")
        label_list.sort()  # Let's sort it for determinism
        num_labels = len(label_list)
        
num_labels, is_regression

(2, False)

In [12]:
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer, PretrainedConfig
model_name_or_path = student
config = AutoConfig.from_pretrained(
    model_name_or_path, 
    num_labels=num_labels, 
    finetuning_task=task_name)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path,
    from_tf=bool(".ckpt" in model_name_or_path),
    config=config,
) #student

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weig

In [37]:
label_to_id = None

if (
    model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id
    and task_name is not None
    and not is_regression
):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if list(sorted(label_name_to_id.keys())) == list(sorted(label_list)):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
        
elif task_name is None and not is_regression:
    label_to_id = {v: i for i, v in enumerate(label_list)}
    
def tokenize_function(examples):
    sentence1_key, sentence2_key = task_to_keys[task_name]
    args = (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
    result = tokenizer(*args, max_length=180, padding="max_length", truncation=True)
    if "label" in examples:
        if label_to_id is not None:
            # Map labels to IDs (not necessary for GLUE tasks)
            result["label"] = [label_to_id[l] for l in examples["label"]]
        else:
            # In all cases, rename the column to labels because the model will expect that.
            result["label"] = examples["label"]
    
    return result

tokenized_datasets = datasets.map(tokenize_function, batched=True)
tokenized_datasets

Loading cached processed dataset at /home/todsavadt/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ddce3eb30c7f5991.arrow
Loading cached processed dataset at /home/todsavadt/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d03df3660f5ad304.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
#Use list comprehension to extract non-None elements from the tuple
elements = [element for element in task_to_keys[task_name] if element is not None]
elements

In [38]:
tokenized_datasets = tokenized_datasets.remove_columns(elements + ["idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tokenized_datasets = tokenized_datasets.remove_columns(elements + ["idx"])                   │
│   2 tokenized_datasets = tokenized_datasets.rename_column("label", "labels")                     │
│   3 tokenized_datasets.set_format("torch")                                                       │
│   4 tokenized_datasets                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'elements' is not defined

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=55) #.select(range(1000))
small_eval_dataset = tokenized_datasets["validation_matched" if task_name == "mnli" else "validation"].shuffle(seed=55)
small_test_dataset = tokenized_datasets["test"].shuffle(seed=55)

## 3. Dataloaders

In [ ]:
from torch.utils.data import DataLoader
per_device_train_batch_size = 16
per_device_eval_batch_size = 16

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=per_device_train_batch_size)
val_dataloader = DataLoader(small_eval_dataset, batch_size=per_device_eval_batch_size)
test_dataloader = DataLoader(small_test_dataset, batch_size=per_device_eval_batch_size)

## 4. Model

In [ ]:
# teacher model
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    teacher,
    config=config,
)

# student model
student_model = AutoModelForSequenceClassification.from_pretrained(
    student,
    config=config,
)
student_model.train()
teacher_model.train()

In [14]:
teacher_checkpoint = 'experiments/best.pth.tar'
checkpoint = utils.load_checkpoint(teacher_checkpoint, model)

In [35]:
# print(list(model.named_modules()))

## 5. Training

## Optimizer 

In [2]:
from torch.optim import AdamW

# Define optimizer
teacher_optimizer = AdamW(teacher_model.parameters(), lr=5e-5)
student_optimizer = AdamW(student_model.parameters(), lr=6e-5)

NameError: name 'teacher_model' is not defined

## Accelerator

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()

teacher_model, teacher_optimizer, train_dataloader, val_dataloader = accelerator.prepare(
    student_model, teacher_optimizer, train_dataloader, val_dataloader
)
# teacher_model = accelerator.prepare(teacher_model)

student_model, student_optimizer, train_dataloader, val_dataloader = accelerator.prepare(
    student_model, student_optimizer, train_dataloader, val_dataloader
)

In [ ]:
from transformers import get_scheduler
import math

gradient_accumulation_steps = 1
num_update_steps_per_epoch = math.ceil(
        len(train_dataloader) / gradient_accumulation_steps
    )

num_train_epochs = 10
max_train_steps = num_train_epochs * num_update_steps_per_epoch

teacher_lr_scheduler = get_scheduler(
    "linear",
    optimizer=teacher_optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

student_lr_scheduler = get_scheduler(
    "linear",
    optimizer=student_optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

total_batch_size = (
        per_device_train_batch_size
        * accelerator.num_processes
        * gradient_accumulation_steps
    )

In [ ]:
### Private Tranformer

In [ ]:
# !pip install ml_swissknife
# !pip install opt_einsum

In [ ]:
import transformers, torch
from private_transformers import PrivacyEngine
dp = True
if dp == True:
    #Student Model
    privacy_engine = PrivacyEngine(
        student_model,
        batch_size=per_device_train_batch_size,
        sample_size=len(datasets['train']),
        epochs=per_device_train_batch_size,
        max_grad_norm=0.1,
        target_epsilon=3,
        clipping_mode="ghost",  # The only change you need to make!
    )
    privacy_engine.attach(student_optimizer)
    #Teacher Model
    privacy_engine = PrivacyEngine(
        teacher_model,
        batch_size=per_device_train_batch_size,
        sample_size=len(datasets['train']),
        epochs=per_device_train_batch_size,
        max_grad_norm=0.1,
        target_epsilon=3,
        clipping_mode="ghost",  # The only change you need to make!
    )
    privacy_engine.attach(teacher_optimizer)
else :
    privacy_engine = None

privacy_engine

In [ ]:
### Loss Objective 


In [3]:
def loss_fn(outputs, labels):
    """
    Compute the cross entropy loss given outputs and labels.

    Returns:
        loss (Variable): cross entropy loss for all images in the batch

    Note: you may use a standard loss function from http://pytorch.org/docs/master/nn.html#loss-functions. This example
          demonstrates how you can easily define a custom loss function.
    """
    return nn.CrossEntropyLoss()(outputs, labels)

## Metrics

In [ ]:
def accuracy(outputs, labels):
    """
    Compute the accuracy, given the outputs and labels for all images.

    Args:
        outputs: (np.ndarray) output of the model
        labels: (np.ndarray) [0, 1, ..., num_classes-1]

    Returns: (float) accuracy in [0,1]
    """
    # outputs = np.argmax(outputs, axis=1)
    return np.sum(outputs==labels)/float(labels.size)


# maintain all metrics required in this dictionary- these are used in the training and evaluation loops
metrics = {
    'accuracy': accuracy,
    # could add more metrics such as accuracy for each token type
}

class RunningAverage():
    """A simple class that maintains the running average of a quantity
    
    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """
    def __init__(self):
        self.steps = 0
        self.total = 0
    
    def update(self, val):
        self.total += val
        self.steps += 1
    
    def __call__(self):
        return self.total/float(self.steps)

In [ ]:
#checking chucking
for i in train_dataloader:
    print(i['input_ids'].shape, i['labels'].shape)
    break
for i in val_dataloader:
    print(i['input_ids'].shape, i['labels'].shape)
    break

In [ ]:
def train(model, optimizer, loss_fn, dataloader, metrics, params):
    # set model to training mode
    model.train()
    
    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = RunningAverage()
    
    for step, batch in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        # batch = tuple(t.to(device) for t in batch)

        # compute model output, fetch teacher output, and compute KD loss
        output_batch = model(**batch) # output = loss, logits, hidden_states, attentions
        labels_batch = batch['labels']

        loss = loss_fn(output_batch, labels_batch)
        loss = loss.reshape(-1)
        # clear previous gradients, compute gradients of all variables wrt loss
        optimizer.zero_grad()
        # loss.backward()
        # accelerator.backward(loss)
        
        # This step is different from existing workflows: 
        # Don't call `loss.backward`; leave it to `optimizer.step` to handle backward.
        # performs updates using calculated gradients
        # `loss` is a 1-D tensor of shape (batch_size,).
        optimizer.step(loss=loss)

        # Evaluate summaries only once in a while
        if step % gradient_accumulation_steps == 0:
            # extract data from torch, move to cpu, convert to numpy arrays            
            output_batch = output_batch.logits.argmax(dim=-1).cpu().numpy()
            labels_batch = labels_batch.cpu().numpy()

            # compute all metrics on this batch
            summary_batch = {metric:metrics[metric](output_batch, labels_batch)
                             for metric in metrics}
            summary_batch['loss'] = loss.item()
            summ.append(summary_batch)

        # update the average loss
        loss_avg.update(loss.item())
        
    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())

In [ ]:
def evaluate(model, loss_fn, dataloader, metrics):
    # set model to evaluation mode
    model.eval()
    # summary for current eval loop
    summ = []

    # compute metrics over the dataset
    for step, batch in enumerate(eval_dataloader):
        
        # compute model output
        output_batch = model(**batch)
        labels_batch = batch['labels']
        
        loss = loss_fn(output_batch, labels_batch)

        # extract data from torch, move to cpu, convert to numpy arrays
        output_batch = output_batch.logits.argmax(dim=-1).cpu().numpy()
        labels_batch = labels_batch.cpu().numpy()

        # compute all metrics on this batch
        summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                         for metric in metrics}
        summary_batch['loss'] = loss.item()
        # summary_batch['loss'] = loss
        summ.append(summary_batch)

    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]} 
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    print("- Eval metrics : " + metrics_string)
    return metrics_mean

In [ ]:
def train_and_evaluate(model, train_dataloader, val_dataloader, optimizer,
                       loss_fn, metrics, model_dir, restore_file=None):
    best_val_acc = 0.0
    for epoch in range(num_train_epochs):
        student_lr_scheduler.step()
        
         # compute number of batches in one epoch (one full pass over the training set)
        train(model, optimizer, loss_fn_kd, train_dataloader,
                 metrics)

        # Evaluate for one epoch on validation set
        val_metrics = evaluate(model, val_dataloader, metrics)

        val_acc = val_metrics['accuracy']
        is_best = val_acc>=best_val_acc
        
        # Save weights
        utils.save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict()},
                               is_best=is_best,
                               checkpoint=model_dir)
        print(f'epoch: {epoch + 1}')
        # If best_eval, best_save_path
        if is_best:
            print("- Found new best accuracy")
            best_val_acc = val_acc

            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(model_dir, "metrics_val_best_weights.json")
            utils.save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(model_dir, "metrics_val_last_weights.json")
        utils.save_dict_to_json(val_metrics, last_json_path)

## Structured DPIMP

![Structured DPIMP](images/Structured_DPIMP.png)

In [ ]:
import numpy as np

def structured_DPIMP(T, L, alpha, N, M):
    S = T.clone()  # Assuming T and S are PyTorch model instances
    
    for j in range(1, L + 1):
        # Fine-tune S for N iterations with DPSGD
        for _ in range(N):
            # Perform DPSGD training here
            train_and_evaluate(model, train_dataloader, val_dataloader, optimizer,
                       loss_fn, metrics, model_dir, restore_file=None):
        
        remaining_weights = []
        for param in S.parameters():
            remaining_weights.extend(param.view(-1).detach().numpy())
        
        # Set Wmin consisting of alpha% of the remaining model weights with the least magnitude
        num_to_keep = int(alpha * len(remaining_weights))
        Wmin_indices = np.argpartition(np.abs(remaining_weights), num_to_keep)[:num_to_keep]
        Wmin = [remaining_weights[i] for i in Wmin_indices]
        
        for i, param in enumerate(S.parameters()):
            Wi = param.view(-1).detach().numpy()
            intersection = np.intersect1d(Wi, Wmin)
            if len(intersection) == 0:
                continue
            
            # Drop the layer i* from S
            i_star = np.argmax([np.sum(Wi == val) for val in intersection])
            # You might want to handle removing the corresponding layer in the model architecture
            
        # Fine-tune S for M more iterations with DPSGD
        for _ in range(M):
            # Perform DPSGD training here
            train_and_evaluate(model, train_dataloader, val_dataloader, optimizer,
                       loss_fn, metrics, model_dir, restore_file=None):
        
    return S

## Unstructured DPIMP

![Unstructured DPIMP](images/Unstructured_DPIMP.png)

In [ ]:
import numpy as np
import torch

def unstructured_DPIMP(T, sparsity, alpha, N, M):
    S = T.clone()  # Assuming T and S are PyTorch model instances
    
    total_params = sum(p.numel() for p in T.parameters())
    num_iterations = int(np.ceil(sparsity / alpha))
    
    T_prime = T.clone()
    
    for i in range(1, num_iterations + 1):
        # Fine-tune T' for N iterations with DPSGD
        for _ in range(N):
            # Perform DPSGD training here
            train_and_evaluate(model, train_dataloader, val_dataloader, optimizer,
                       loss_fn, metrics, model_dir, restore_file=None):
        
        # Prune (alpha * i)% of weights with the lowest magnitude from T'
        num_params_to_prune = int(alpha * i * total_params)
        all_weights = np.concatenate([param.view(-1).detach().numpy() for param in T_prime.parameters()])
        sorted_indices = np.argsort(np.abs(all_weights))
        prune_indices = sorted_indices[:num_params_to_prune]
        
        for param in T_prime.parameters():
            flat_param = param.view(-1).detach().numpy()
            flat_param[prune_indices] = 0
            param.data = torch.from_numpy(flat_param).reshape(param.shape)
        
        # Reset the non-zero weights of T' to the original T
        for param_s, param_t in zip(S.parameters(), T.parameters()):
            if param_t.requires_grad:
                param_s.data = param_t.data.clone()
        
    # Fine-tune T' for M iterations with DPSGD
    for _ in range(M):
        # Perform DPSGD training here
        train_and_evaluate(model, train_dataloader, val_dataloader, optimizer,
                       loss_fn, metrics, model_dir, restore_file=None):
        
    return S